# Kaggleで磨く 機械学習の実践力
# 第6章 モデルチューニング

# 6.1 LightGBMのハイパーパラメータのチューニング
## 6.1.2 ハイパーパラメータの自動チューニング

#### スクリプト: ライブラリのインポート (スクリプト4-1の再掲)

In [1]:
# 2022/06/02追加: Kaggle notebook環境変更のため
!pip install pandas_profiling==3.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.6/262.6 kB 853.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: visions
    Found existing installation: visions 0.7.5
    Uninstalling visions-0.7.5:
      Successfully uninstalled visions-0.7.5
  Attempting uninstall: pandas_profiling
    Found existing installation: pandas-profiling 2.4.0
    Uninstalling pandas-profiling-2.4.0:
      Successfully uninstalled pandas-profiling-2.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.


In [2]:
import numpy as np
import pandas as pd
import os
import pickle
import gc

# 分布確認
import pandas_profiling as pdp

# 可視化
import matplotlib.pyplot as plt

# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

# バリデーション
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold

# 評価指標
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

# モデリング: lightgbm
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

#### スクリプト: ファイルの読み込み (スクリプト4-2の再掲)

In [3]:
df_train = pd.read_csv("../input/titanic/train.csv")

#### スクリプト: データセット作成 (スクリプト4-8の再掲)

In [4]:
x_train, y_train, id_train = df_train[["Pclass", "Fare"]], \
                             df_train[["Survived"]], \
                             df_train[["PassengerId"]]
print(x_train.shape, y_train.shape, id_train.shape)

(891, 2) (891, 1) (891, 1)


#### スクリプト6-1: optunaのインポート

In [5]:
import optuna

#### スクリプト6-2: 目的関数の定義

In [6]:
# 探索しないハイパーパラメータ
params_base = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.02,
    'n_estimators': 100000,
    "bagging_freq": 1,
    "seed": 123,
}

def objective(trial):
    # 探索するハイパーパラメータ
    params_tuning = {
        "num_leaves": trial.suggest_int("num_leaves", 8, 256),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 200),
        "min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e2, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e2, log=True),
    }
    params_tuning.update(params_base)
    
    # モデル学習・評価
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    for nfold in np.arange(5):
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train.loc[idx_tr, :]
        x_va, y_va = x_train.loc[idx_va, :], y_train.loc[idx_va, :]
        model = lgb.LGBMClassifier(**params_tuning)
        model.fit(x_tr,
                  y_tr,
                  eval_set=[(x_tr,y_tr), (x_va,y_va)],
                  early_stopping_rounds=100,
                  verbose=0,
                 )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = accuracy_score(y_va, np.where(y_va_pred>=0.5, 1, 0))
        list_metrics.append(metric_va)
    
    # 評価値の計算
    metrics = np.mean(list_metrics)
    
    return metrics

#### スクリプト6-3: 最適化処理（探索の実行）

In [7]:
sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction="maximize")
study.optimize(objective, n_trials=30)

[I 2022-06-02 02:04:53,682] A new study created in memory with name: no-name-ef3aec94-1d5f-4a33-bb68-d40d38ac945d


[LightGBM] [Warning] lambda_l1 is set=0.492522233779106, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.492522233779106
[LightGBM] [Warning] bagging_fraction is set=0.8597344848927815, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8597344848927815
[LightGBM] [Warning] lambda_l2 is set=83.76388146302445, reg_lambda=0.0 will be ignored. Current value: lambda_l2=83.76388146302445
[LightGBM] [Warning] feature_fraction is set=0.7756573845414456, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7756573845414456
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=4.792414358623587e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=4.792414358623587e-05
[LightGBM] [Warning] min_data_in_leaf is set=61, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=61
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1

[I 2022-06-02 02:04:54,344] Trial 0 finished with value: 0.664478061640826 and parameters: {'num_leaves': 181, 'min_data_in_leaf': 61, 'min_sum_hessian_in_leaf': 4.792414358623587e-05, 'feature_fraction': 0.7756573845414456, 'bagging_fraction': 0.8597344848927815, 'lambda_l1': 0.492522233779106, 'lambda_l2': 83.76388146302445}. Best is trial 0 with value: 0.664478061640826.


[LightGBM] [Warning] lambda_l1 is set=0.567922374174008, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.567922374174008
[LightGBM] [Warning] bagging_fraction is set=0.8645248536920208, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8645248536920208
[LightGBM] [Warning] lambda_l2 is set=0.01732652966363563, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01732652966363563
[LightGBM] [Warning] feature_fraction is set=0.6715890080754348, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6715890080754348
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.00015009027543233888, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.00015009027543233888
[LightGBM] [Warning] min_data_in_leaf is set=99, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=99
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lam

[I 2022-06-02 02:04:54,723] Trial 1 finished with value: 0.6712196346745339 and parameters: {'num_leaves': 178, 'min_data_in_leaf': 99, 'min_sum_hessian_in_leaf': 0.00015009027543233888, 'feature_fraction': 0.6715890080754348, 'bagging_fraction': 0.8645248536920208, 'lambda_l1': 0.567922374174008, 'lambda_l2': 0.01732652966363563}. Best is trial 1 with value: 0.6712196346745339.


...

[LightGBM] [Warning] lambda_l1 is set=0.132136979057582, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.132136979057582
[LightGBM] [Warning] bagging_fraction is set=0.9245894599408239, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9245894599408239
[LightGBM] [Warning] lambda_l2 is set=1.9898087013583583, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.9898087013583583
[LightGBM] [Warning] feature_fraction is set=0.8328590281541173, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8328590281541173
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.0011015177430549294, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.0011015177430549294
[LightGBM] [Warning] min_data_in_leaf is set=110, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=110
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambd

[I 2022-06-02 02:05:06,577] Trial 28 finished with value: 0.6161634548992531 and parameters: {'num_leaves': 54, 'min_data_in_leaf': 169, 'min_sum_hessian_in_leaf': 0.00044638503697018715, 'feature_fraction': 0.7818076921940363, 'bagging_fraction': 0.7239926057056618, 'lambda_l1': 0.26894380562875136, 'lambda_l2': 82.43363076639685}. Best is trial 7 with value: 0.6992530286862093.


[LightGBM] [Warning] lambda_l1 is set=0.26894380562875136, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.26894380562875136
[LightGBM] [Warning] bagging_fraction is set=0.7239926057056618, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7239926057056618
[LightGBM] [Warning] lambda_l2 is set=82.43363076639685, reg_lambda=0.0 will be ignored. Current value: lambda_l2=82.43363076639685
[LightGBM] [Warning] feature_fraction is set=0.7818076921940363, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7818076921940363
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.00044638503697018715, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.00044638503697018715
[LightGBM] [Warning] min_data_in_leaf is set=169, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=169
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] l

[I 2022-06-02 02:05:06,884] Trial 29 finished with value: 0.6587470968551881 and parameters: {'num_leaves': 161, 'min_data_in_leaf': 196, 'min_sum_hessian_in_leaf': 8.505644215173895e-05, 'feature_fraction': 0.7116385045852216, 'bagging_fraction': 0.9850169158486759, 'lambda_l1': 0.022763155034800718, 'lambda_l2': 32.318206202200265}. Best is trial 7 with value: 0.6992530286862093.


[LightGBM] [Warning] lambda_l1 is set=0.022763155034800718, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.022763155034800718
[LightGBM] [Warning] bagging_fraction is set=0.9850169158486759, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9850169158486759
[LightGBM] [Warning] lambda_l2 is set=32.318206202200265, reg_lambda=0.0 will be ignored. Current value: lambda_l2=32.318206202200265
[LightGBM] [Warning] feature_fraction is set=0.7116385045852216, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7116385045852216
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8.505644215173895e-05, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=8.505644215173895e-05
[LightGBM] [Warning] min_data_in_leaf is set=196, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=196
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning]

#### スクリプト6-4: 探索結果の確認

In [8]:
trial = study.best_trial
print("acc(best)={:.4f}".format(trial.value))
display(trial.params)

acc(best)=0.6993


{'num_leaves': 160,
 'min_data_in_leaf': 28,
 'min_sum_hessian_in_leaf': 0.0030131614432849746,
 'feature_fraction': 0.8015300642054637,
 'bagging_fraction': 0.7725340032332324,
 'lambda_l1': 0.23499322154972468,
 'lambda_l2': 0.1646202117975735}

#### スクリプト6-5: ベストなハイパーパラメータの取得

In [9]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

{'num_leaves': 160,
 'min_data_in_leaf': 28,
 'min_sum_hessian_in_leaf': 0.0030131614432849746,
 'feature_fraction': 0.8015300642054637,
 'bagging_fraction': 0.7725340032332324,
 'lambda_l1': 0.23499322154972468,
 'lambda_l2': 0.1646202117975735,
 'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'learning_rate': 0.02,
 'n_estimators': 100000,
 'bagging_freq': 1,
 'seed': 123}

# 6.2 LightGBM以外のモデル利用
## 6.2.1 scikit-learnの各種モデル

### Titanicデータを用いた例：ロジスティック回帰
#### スクリプト6-6: ファイル読み込みとデータセット作成

In [10]:
# ファイル読み込み
df_train = pd.read_csv("../input/titanic/train.csv")

# データセット作成
x_train = df_train[["Pclass", "Age", "Embarked"]]
y_train = df_train[["Survived"]]

In [11]:
# 欠損値の確認
x_train.isnull().sum()

Pclass        0
Age         177
Embarked      2
dtype: int64

#### スクリプト6-7: 欠損値の補間

In [12]:
# 欠損値補間：数値データ
x_train["Age"] = x_train["Age"].fillna(x_train["Age"].mean())

# 欠損値補間：カテゴリ変数
x_train["Embarked"] = x_train["Embarked"].fillna(x_train["Embarked"].mode()[0])

#### スクリプト6-8: カテゴリ変数の数値化（one-hot-encoding）

In [13]:
ohe = OneHotEncoder()
ohe.fit(x_train[["Embarked"]])
df_embarked = pd.DataFrame(
    ohe.transform(x_train[["Embarked"]]).toarray(), 
    columns=["Embarked_{}".format(col) for col in ohe.categories_[0]])

x_train = pd.concat([x_train, df_embarked], axis=1)
x_train = x_train.drop(columns=["Embarked"])

#### スクリプト6-9: 数値データの正規化

In [14]:
x_train["Pclass"] = (x_train["Pclass"] -x_train["Pclass"].min()) / (x_train["Pclass"].max() - x_train["Pclass"].min()) 
x_train["Age"] = (x_train["Age"] -x_train["Age"].min()) / (x_train["Age"].max() - x_train["Age"].min()) 

#### スクリプト6-10: 学習データと検証データの分割（ホールドアウト検証）

In [15]:
x_tr, x_va, y_tr, y_va = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=123)
print(x_tr.shape, x_va.shape, y_tr.shape, y_va.shape)

(712, 5) (179, 5) (712, 1) (179, 1)


#### スクリプト6-11: LogisticRegression

In [16]:
# モデル定義
from sklearn.linear_model import LogisticRegression
model_logis = LogisticRegression()

# 学習
model_logis.fit(x_tr, y_tr)

# 予測
y_va_pred = model_logis.predict(x_va)
print("accuracy:{:.4f}".format(accuracy_score(y_va, y_va_pred)))
print(y_va_pred[:5])

accuracy:0.7263
[0 1 0 1 0]


#### スクリプト6-12: 確率値の取得

In [17]:
y_va_pred_proba = model_logis.predict_proba(x_va)
print(y_va_pred_proba[:5, :])

[[0.83621285 0.16378715]
 [0.23058311 0.76941689]
 [0.83244141 0.16755859]
 [0.32227072 0.67772928]
 [0.62569522 0.37430478]]


### Titanicデータを用いた例：SVM
#### スクリプト6-13: SVM

In [18]:
# モデル定義
from sklearn.svm import SVC
model_svm = SVC(C=1.0, random_state=123, probability=True)

# 学習
model_svm.fit(x_tr, y_tr)

# 予測
y_va_pred = model_svm.predict(x_va)
print("accuracy:{:.4f}".format(accuracy_score(y_va, y_va_pred)))
print(y_va_pred[:5])

# 確率値の取得
y_va_pred_proba = model_svm.predict_proba(x_va)
print(y_va_pred_proba[:5, :])

accuracy:0.7151
[0 1 0 1 0]
[[0.73985924 0.26014076]
 [0.28242534 0.71757466]
 [0.73986177 0.26013823]
 [0.26828214 0.73171786]
 [0.58950192 0.41049808]]


## 6.2.2 ニューラルネットワーク


### ニューラルネットワークの適用例：①全結合層のみのネットワークモデル
#### スクリプト6-14: ライブラリのインポート

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Embedding, Flatten, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import Adam, SGD

#### スクリプト6-15: tensorflowの再現性のためのシード指定

In [20]:
def seed_everything(seed):
    import random
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

#### スクリプト6-16: ファイルの読み込みとデータセット作成

In [21]:
# ファイル読み込み
df_train = pd.read_csv("../input/titanic/train.csv")

# データセット作成
x_train = df_train[["Pclass", "Age", "Embarked"]]
y_train = df_train[["Survived"]]

#### スクリプト6-17: 数値データの前処理

In [22]:
# 欠損値補間
x_train["Age"] = x_train["Age"].fillna(x_train["Age"].mean())

# 正規化
for col in ["Pclass", "Age"]:
    value_min = x_train[col].min()
    value_max = x_train[col].max()
    x_train[col] = (x_train[col] - value_min) / (value_max - value_min)

#### スクリプト6-18: カテゴリ変数の前処理

In [23]:
# 欠損値補間
x_train["Embarked"] = x_train["Embarked"].fillna(x_train["Embarked"].mode()[0])

# one-hot-encoding
ohe = OneHotEncoder()
ohe.fit(x_train[["Embarked"]])
df_embarked = pd.DataFrame(ohe.transform(x_train[["Embarked"]]).toarray(), 
                           columns=["Embarked_{}".format(col) for col in ohe.categories_[0]])
x_train = pd.concat([x_train.drop(columns=["Embarked"]), 
                     df_embarked], axis=1)

#### スクリプト6-19: 学習データと検証データの分割

In [24]:
x_tr, x_va, y_tr, y_va = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=123)
print(x_tr.shape, x_va.shape, y_tr.shape, y_va.shape)

(712, 5) (179, 5) (712, 1) (179, 1)


#### スクリプト6-20: モデル定義

In [25]:
def create_model():
    input_num = Input(shape=(5,))
    x_num = Dense(10, activation="relu")(input_num)
    x_num = BatchNormalization()(x_num)
    x_num = Dropout(0.3)(x_num)
    x_num = Dense(10, activation="relu")(x_num)
    x_num = BatchNormalization()(x_num)
    x_num = Dropout(0.2)(x_num)
    x_num = Dense(5, activation="relu")(x_num)
    x_num = BatchNormalization()(x_num)
    x_num = Dropout(0.1)(x_num)
    out = Dense(1, activation="sigmoid")(x_num)

    model = Model(inputs=input_num,
                  outputs=out,
                 )

    model.compile(
        optimizer="Adam",
        loss="binary_crossentropy",
        metrics=["binary_crossentropy"],
    )
    
    return model

model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense (Dense)                (None, 10)                60        
_________________________________________________________________
batch_normalization (BatchNo (None, 10)                40        
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
batch_normalization_1 (Batch (None, 10)                40        
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0     

2022-06-02 02:05:16.064511: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


#### スクリプト6-21: モデル学習

In [26]:
seed_everything(seed=123)
model = create_model()
model.fit(x=x_tr,
          y=y_tr,
          validation_data=(x_va, y_va),
          batch_size=8,
          epochs=10000,
          callbacks=[
              ModelCheckpoint(filepath="model_keras.h5", monitor="val_loss", mode="min", verbose=1, save_best_only=True, save_weights_only=True),
              EarlyStopping(monitor="val_loss", mode="min", min_delta=0, patience=10, verbose=1, restore_best_weights=True),
              ReduceLROnPlateau(monitor="val_loss", mode="min", factor=0.1, patience=5, verbose=1),
          ],
          verbose=1,
         )

2022-06-02 02:05:17.163928: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10000
89/89 [==============================] - 2s 7ms/step - loss: 0.6693 - binary_crossentropy: 0.6693 - val_loss: 0.6570 - val_binary_crossentropy: 0.6570

Epoch 00001: val_loss improved from inf to 0.65698, saving model to model_keras.h5
Epoch 2/10000
89/89 [==============================] - 0s 3ms/step - loss: 0.6714 - binary_crossentropy: 0.6714 - val_loss: 0.6354 - val_binary_crossentropy: 0.6354

Epoch 00002: val_loss improved from 0.65698 to 0.63545, saving model to model_keras.h5
Epoch 3/10000
89/89 [==============================] - 0s 3ms/step - loss: 0.6606 - binary_crossentropy: 0.6606 - val_loss: 0.6215 - val_binary_crossentropy: 0.6215

Epoch 00003: val_loss improved from 0.63545 to 0.62146, saving model to model_keras.h5
Epoch 4/10000
89/89 [==============================] - 0s 3ms/step - loss: 0.6462 - binary_crossentropy: 0.6462 - val_loss: 0.6120 - val_binary_crossentropy: 0.6120

Epoch 00004: val_loss improved from 0.62146 to 0.61204, saving model to model_k

#### スクリプト6-22: モデルの評価

In [27]:
y_va_pred = model.predict(x_va, batch_size=8, verbose=1)
print("accuracy: {:.4f}".format(accuracy_score(y_va, np.where(y_va_pred>=0.5,1,0))))

23/23 [==============================] - 0s 1ms/step
accuracy: 0.7151


### ニューラルネットワークの適用例：②埋め込み層ありのネットワークモデル
#### スクリプト6-23: ファイルの読み込みとデータセット作成

In [28]:
# ファイル読み込み
df_train = pd.read_csv("../input/titanic/train.csv")

# データセット作成
x_train = df_train[["Pclass", "Age", "Cabin"]]
y_train = df_train[["Survived"]]

#### スクリプト6-24: 数値データの前処理

In [29]:
# 欠損値補間
x_train["Age"] = x_train["Age"].fillna(x_train["Age"].mean())

# 正規化
for col in ["Pclass", "Age"]:
    value_min = x_train[col].min()
    value_max = x_train[col].max()
    x_train[col] = (x_train[col] - value_min) / (value_max - value_min)

#### スクリプト6-25: カテゴリ変数の前処理

In [30]:
# 欠損値補間
x_train["Cabin"] = x_train["Cabin"].fillna("None")

# label-encoding
le = LabelEncoder()
le.fit(x_train[["Cabin"]])
x_train["Cabin"] = le.transform(x_train["Cabin"])

print(le.classes_)
print("count:", len(le.classes_))

['A10' 'A14' 'A16' 'A19' 'A20' 'A23' 'A24' 'A26' 'A31' 'A32' 'A34' 'A36'
 'A5' 'A6' 'A7' 'B101' 'B102' 'B18' 'B19' 'B20' 'B22' 'B28' 'B3' 'B30'
 'B35' 'B37' 'B38' 'B39' 'B4' 'B41' 'B42' 'B49' 'B5' 'B50' 'B51 B53 B55'
 'B57 B59 B63 B66' 'B58 B60' 'B69' 'B71' 'B73' 'B77' 'B78' 'B79' 'B80'
 'B82 B84' 'B86' 'B94' 'B96 B98' 'C101' 'C103' 'C104' 'C106' 'C110' 'C111'
 'C118' 'C123' 'C124' 'C125' 'C126' 'C128' 'C148' 'C2' 'C22 C26'
 'C23 C25 C27' 'C30' 'C32' 'C45' 'C46' 'C47' 'C49' 'C50' 'C52' 'C54'
 'C62 C64' 'C65' 'C68' 'C7' 'C70' 'C78' 'C82' 'C83' 'C85' 'C86' 'C87'
 'C90' 'C91' 'C92' 'C93' 'C95' 'C99' 'D' 'D10 D12' 'D11' 'D15' 'D17' 'D19'
 'D20' 'D21' 'D26' 'D28' 'D30' 'D33' 'D35' 'D36' 'D37' 'D45' 'D46' 'D47'
 'D48' 'D49' 'D50' 'D56' 'D6' 'D7' 'D9' 'E10' 'E101' 'E12' 'E121' 'E17'
 'E24' 'E25' 'E31' 'E33' 'E34' 'E36' 'E38' 'E40' 'E44' 'E46' 'E49' 'E50'
 'E58' 'E63' 'E67' 'E68' 'E77' 'E8' 'F E69' 'F G63' 'F G73' 'F2' 'F33'
 'F38' 'F4' 'G6' 'None' 'T']
count: 148


#### スクリプト6-26: 学習データと検証データの分離

In [31]:
x_train_num, x_train_cat = x_train[["Pclass", "Age"]], x_train[["Cabin"]]

x_num_tr, x_num_va, x_cat_tr, x_cat_va, y_tr, y_va = \
   train_test_split(x_train_num, x_train_cat, y_train, test_size=0.2, stratify=y_train, random_state=123)
print(x_num_tr.shape, x_num_va.shape, x_cat_tr.shape, x_cat_va.shape, y_tr.shape, y_va.shape)

(712, 2) (179, 2) (712, 1) (179, 1) (712, 1) (179, 1)


#### スクリプト6-27: モデル定義

In [32]:
def create_model_embedding():
    ################# num
    input_num = Input(shape=(2,))
    layer_num = Dense(10, activation="relu")(input_num)
    layer_num = BatchNormalization()(layer_num)
    layer_num = Dropout(0.2)(layer_num)
    layer_num = Dense(10, activation="relu")(layer_num)

    ################# cat
    input_cat = Input(shape=(1,))
    layer_cat = input_cat[:, 0]
    layer_cat = Embedding(input_dim=148, output_dim=74)(layer_cat)
    layer_cat = Dropout(0.2)(layer_cat)
    layer_cat = Flatten()(layer_cat)

    ################# concat
    hidden_layer = Concatenate()([layer_num, layer_cat])
    hidden_layer = Dense(50, activation="relu")(hidden_layer)
    hidden_layer = BatchNormalization()(hidden_layer)
    hidden_layer = Dropout(0.1)(hidden_layer)
    hidden_layer = Dense(20, activation="relu")(hidden_layer)
    hidden_layer = BatchNormalization()(hidden_layer)
    hidden_layer = Dropout(0.1)(hidden_layer)
    output_layer = Dense(1, activation="sigmoid")(hidden_layer)

    model = Model(inputs=[input_num, input_cat],
                  outputs=output_layer,
                 )

    model.compile(
        optimizer="Adam",
        loss="binary_crossentropy",
        metrics=["binary_crossentropy"],
    )
    
    return model

model = create_model_embedding()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 10)           30          input_3[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None,)              0           input_4[0][0]                    
____________________________________________________________________________________________

#### スクリプト6-28: モデルの学習

In [33]:
seed_everything(seed=123)
model = create_model_embedding()
model.fit(x=[x_num_tr, x_cat_tr],
          y=y_tr,
          validation_data=([x_num_va, x_cat_va], y_va),
          batch_size=8,
          epochs=10000,
          callbacks=[
              ModelCheckpoint(filepath="model_keras_embedding.h5", monitor="val_loss", mode="min", verbose=1, save_best_only=True, save_weights_only=True),
              EarlyStopping(monitor="val_loss", mode="min", min_delta=0, patience=10, verbose=1, restore_best_weights=True),
              ReduceLROnPlateau(monitor="val_loss", mode="min", factor=0.1, patience=5, verbose=1),
          ],
          verbose=1,
         )

Epoch 1/10000
89/89 [==============================] - 2s 6ms/step - loss: 0.7839 - binary_crossentropy: 0.7839 - val_loss: 0.7185 - val_binary_crossentropy: 0.7185

Epoch 00001: val_loss improved from inf to 0.71853, saving model to model_keras_embedding.h5
Epoch 2/10000
89/89 [==============================] - 0s 3ms/step - loss: 0.6714 - binary_crossentropy: 0.6714 - val_loss: 0.7027 - val_binary_crossentropy: 0.7027

Epoch 00002: val_loss improved from 0.71853 to 0.70270, saving model to model_keras_embedding.h5
Epoch 3/10000
89/89 [==============================] - 0s 3ms/step - loss: 0.6603 - binary_crossentropy: 0.6603 - val_loss: 0.6851 - val_binary_crossentropy: 0.6851

Epoch 00003: val_loss improved from 0.70270 to 0.68515, saving model to model_keras_embedding.h5
Epoch 4/10000
89/89 [==============================] - 0s 3ms/step - loss: 0.6500 - binary_crossentropy: 0.6500 - val_loss: 0.6301 - val_binary_crossentropy: 0.6301

Epoch 00004: val_loss improved from 0.68515 to 0.

#### スクリプト6-29: モデル評価

In [34]:
y_va_pred = model.predict([x_num_va, x_cat_va], batch_size=8, verbose=1)
print("accuracy: {:.4f}".format(accuracy_score(y_va, np.where(y_va_pred>=0.5,1,0))))

23/23 [==============================] - 0s 1ms/step
accuracy: 0.7151


# 6.3 アンサンブル
## 6.3.1 単純平均

#### スクリプト6-30: 3モデルの予測値を持つデータフレームを乱数で作成

In [35]:
np.random.seed(123)
df = pd.DataFrame({
    "true": [0]*700 + [1]*300,
    "pred1":np.arange(1000) + np.random.rand(1000)*1200,
    "pred2":np.arange(1000) + np.random.rand(1000)*1000,
    "pred3":np.arange(1000) + np.random.rand(1000)*800,
})
df["pred1"] = np.clip(df["pred1"]/df["pred1"].max(), 0, 1)
df["pred2"] = np.clip(df["pred2"]/df["pred2"].max(), 0, 1)
df["pred3"] = np.clip(df["pred3"]/df["pred3"].max(), 0, 1)

df_train, df_test = train_test_split(df, test_size=0.8, stratify=df["true"], random_state=123)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.head()

,true,pred1,pred2,pred3
0,1,0.683821,0.874443,0.859939
1,0,0.540691,0.113419,0.197144
2,0,0.310541,0.334798,0.599304
3,0,0.043486,0.170622,0.378528
4,0,0.550847,0.354703,0.598860


#### スクリプト6-31: 単純平均によるアンサンブル

In [36]:
df_train["pred_ensemble1"] = (df_train["pred1"] + df_train["pred2"] + df_train["pred3"]) / 3
df_train.head()

,true,pred1,pred2,pred3,pred_ensemble1
0,1,0.683821,0.874443,0.859939,0.806068
1,0,0.540691,0.113419,0.197144,0.283752
2,0,0.310541,0.334798,0.599304,0.414881
3,0,0.043486,0.170622,0.378528,0.197545
4,0,0.550847,0.354703,0.598860,0.501470


#### スクリプト6-32: アンサンブル用の精度評価関数と、精度評価

In [37]:
def evaluate_ensemble(input_df, col_pred):
    print("[auc] model1:{:.4f}, model2:{:.4f}, model3:{:.4f} -> ensemble:{:.4f}".format(
        roc_auc_score(input_df["true"], input_df["pred1"]),
        roc_auc_score(input_df["true"], input_df["pred2"]),
        roc_auc_score(input_df["true"], input_df["pred3"]),
        roc_auc_score(input_df["true"], input_df[col_pred]),
    ))

evaluate_ensemble(df_train, col_pred="pred_ensemble1")

[auc] model1:0.8342, model2:0.8671, model3:0.9050 -> ensemble:0.9585


#### スクリプト6-33: 推論時のアンサンブル処理と、精度評価

In [38]:
df_test["pred_ensemble1"] = (df_test["pred1"] + df_test["pred2"] + df_test["pred3"]) / 3
evaluate_ensemble(df_test, col_pred="pred_ensemble1")

[auc] model1:0.8086, model2:0.8398, model3:0.8973 -> ensemble:0.9396


## 6.3.2 重み付き平均

#### スクリプト6-34: 重み付き平均によるアンサンブル

In [39]:
weight = [0.3, 0.3, 0.4]
weight = weight / np.sum(weight)
print(weight)

df_train["pred_ensemble2"] = df_train["pred1"] * weight[0] + \
                             df_train["pred2"] * weight[1] + \
                             df_train["pred3"] * weight[2]
df_train[["true","pred1","pred2","pred3","pred_ensemble2"]].head()

[0.3 0.3 0.4]


,true,pred1,pred2,pred3,pred_ensemble2
0,1,0.683821,0.874443,0.859939,0.811455
1,0,0.540691,0.113419,0.197144,0.275091
2,0,0.310541,0.334798,0.599304,0.433324
3,0,0.043486,0.170622,0.378528,0.215643
4,0,0.550847,0.354703,0.598860,0.511209


#### スクリプト6-35: アンサンブルの精度評価

In [40]:
evaluate_ensemble(df_train, col_pred="pred_ensemble2")

[auc] model1:0.8342, model2:0.8671, model3:0.9050 -> ensemble:0.9614


#### スクリプト6-36: 推論時のアンサンブル処理と、精度評価

In [41]:
df_test["pred_ensemble2"] = df_test["pred1"] * weight[0] + \
                            df_test["pred2"] * weight[1] + \
                            df_test["pred3"] * weight[2]
evaluate_ensemble(df_test, col_pred="pred_ensemble2")

[auc] model1:0.8086, model2:0.8398, model3:0.8973 -> ensemble:0.9420


## 6.3.3 スタッキング

#### スクリプト6-37: スタッキングによるアンサンブル

In [42]:
from sklearn.linear_model import Lasso

x, y = df_train[["pred1", "pred2", "pred3"]], df_train[["true"]]
oof = np.zeros(len(x))
models = []

cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x, y))
for nfold in np.arange(5):
    # 学習データと検証データの分離
    idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
    x_tr, y_tr = x.loc[idx_tr, :], y.loc[idx_tr, :]
    x_va, y_va = x.loc[idx_va, :], y.loc[idx_va, :]
    
    # モデル学習
    model = Lasso(alpha=0.01)
    model.fit(x_tr, y_tr)
    models.append(model)
    
    # 検証データの予測値算出
    y_va_pred = model.predict(x_va)
    oof[idx_va] = y_va_pred
    
df_train["pred_ensemble3"] = oof
df_train["pred_ensemble3"] = df_train["pred_ensemble3"].clip(lower=0, upper=1)
df_train[["true","pred1","pred2","pred3","pred_ensemble3"]].head()

,true,pred1,pred2,pred3,pred_ensemble3
0,1,0.683821,0.874443,0.859939,0.745020
1,0,0.540691,0.113419,0.197144,0.000000
2,0,0.310541,0.334798,0.599304,0.206734
3,0,0.043486,0.170622,0.378528,0.000000
4,0,0.550847,0.354703,0.598860,0.303498


#### スクリプト6-38: アンサンブルの精度評価

In [43]:
evaluate_ensemble(df_train, col_pred="pred_ensemble3")

[auc] model1:0.8342, model2:0.8671, model3:0.9050 -> ensemble:0.9577


#### スクリプト6-39: 推論時のアンサンブル処理と、精度評価

In [44]:
df_test["pred_ensemble3"] = 0
for model in models:
    df_test["pred_ensemble3"] += model.predict(df_test[["pred1", "pred2", "pred3"]]) / len(models)
df_test["pred_ensemble3"] = df_test["pred_ensemble3"].clip(lower=0, upper=1)
evaluate_ensemble(df_test, col_pred="pred_ensemble3")

[auc] model1:0.8086, model2:0.8398, model3:0.8973 -> ensemble:0.9437
